In [1]:
import pandas as pd

In [2]:
db = pd.read_csv("/Users/damoncrockett/Dropbox/lml/export_06_18_24.csv", encoding='latin-1')

In [3]:
db = db.fillna('')
db['catalog'] = db['Catalog Number'].apply(str) + db['Secondary Catalog Number'].apply(lambda x: x.lower())

In [4]:
db = db.loc[db.catalog!='2523']

In [5]:
db = db.reset_index(drop=True)

# sbi

In [7]:
db.Exposure.value_counts()

Exposure
        2712
9.0      312
10.0     311
8.0      246
6.0      236
7.0      234
5.0      225
4.0      183
11.0     179
13.0     179
12.0     177
3.0      176
14.0     171
15.0     146
16.0     125
17.0     119
19.0     117
18.0     113
2.0      107
20.0      97
21.0      73
22.0      65
23.0      65
25.0      61
24.0      56
26.0      49
27.0      46
29.0      42
35.0      41
28.0      38
33.0      38
31.0      35
34.0      35
32.0      35
30.0      34
38.0      34
40.0      31
41.0      30
37.0      26
36.0      23
1.0       23
44.0      20
42.0      18
39.0      16
43.0      15
48.0       9
46.0       8
47.0       6
0.0        6
45.0       6
49.0       5
50.0       5
51.0       4
52.0       3
53.0       2
57.0       1
59.0       1
61.0       1
54.0       1
Name: count, dtype: int64

# img

In [7]:
import glob,os

In [8]:
pkgs = glob.glob(os.path.join("/Users/damoncrockett/paperbase/src/assets/img/packages/","*.jpg"))
sbs = glob.glob(os.path.join("/Users/damoncrockett/paperbase/src/assets/img/samplebooks/","*.jpeg"))

In [9]:
pf = pd.DataFrame({"localpath":pkgs})
sf = pd.DataFrame({"localpath":sbs})

In [10]:
import re

def process_string(input_string):
    # Check if the string contains an underscore
    if "_" in input_string:
        return None
    
    # Check for any letters in the string
    if re.search("[a-zA-Z]", input_string):
        # Separate numbers from letters
        numbers = re.sub("[^0-9]", "", input_string)
        letters = re.sub("[^a-zA-Z]", "", input_string)
        return [numbers, letters]
    
    # Return the input string if there are no letters (only numbers)
    return [input_string]

In [11]:
procstr = [process_string(os.path.basename(item)[:-4]) for item in pf.localpath]

In [12]:
pf['procstr'] = procstr

In [13]:
pf['catalog'] = [item[0] if item is not None else item for item in pf.procstr]

In [14]:
pf = pf.loc[pf.procstr.notnull()]

In [15]:
pf = pf.groupby('catalog')['localpath'].agg(list).reset_index()

In [16]:
#smp = pf.sample(n=100)

In [17]:
tmp = pd.read_pickle('smp.pkl')

In [20]:
cats = tmp.catalog
smp = pf.loc[pf.catalog.isin(cats)]

In [21]:
smp =smp.join(db.set_index('catalog'),'catalog')

In [22]:
#smp.to_pickle('smp.pkl')

In [23]:
# try:
#     os.mkdir("/Users/damoncrockett/lml2lux/iiif_smp/")
# except Exception as e:
#     print(e)

In [24]:
#COPYDIR = "/Users/damoncrockett/lml2lux/iiif_smp/"

In [25]:
#allimg = [item for sublist in smp.localpath for item in sublist]

In [26]:
#import shutil

In [27]:
# for img in allimg:
#     shutil.copy(img,COPYDIR)

In [29]:
smp.columns = ['ObjectNumber','ImagePaths','Manufacturer','Brand','Year','Format','DateUncertain']

In [30]:
smp['CreditLine'] = 'Lens Media Lab Collection'

In [31]:
smp['Classification'] = "photographic paper package"

In [32]:
smp['Department'] = "Lens Media Lab"

In [33]:
smp['Institution'] = "Institute for the Preservation of Cultural Heritage"

In [34]:
smp['Date'] = [f'ca. {smp.Year.loc[i]}' if smp.DateUncertain.loc[i]==1 else str(smp.Year.loc[i]) for i in smp.index]

In [35]:
smp['ObjectLabel'] = [f'{smp.Manufacturer.loc[i]} {smp.Brand.loc[i]} {smp.Date.loc[i]}' for i in smp.index]

In [36]:
del smp['Format']

In [37]:
del smp['DateUncertain']

In [38]:
def get_image_object(p):
    imgid = p[:-4]
    imgfile = p
    ps = process_string(imgid)
    if len(ps)==1:
        imglabel = "recto"
        imgrank = "1"
    elif len(ps)==2:
        v = ps[1]
        if v == 'v':
            imglabel = 'verso'
            imgrank = "2"
        elif v == 'l':
            imglabel = 'left'
            imgrank = "3"
        elif v == 'r':
            imglabel = 'right'
            imgrank = "4"
        elif v == 't':
            imglabel = 'top'
            imgrank = "5"
        elif v == 'b':
            imglabel = 'bottom'
            imgrank = "0"
        else:
            print(v)

    o = {}
    o['id'] = imgid
    o['image_filename'] = imgfile
    o['image_label'] = imglabel
    o['rank'] = imgrank
    o['object_thumbnail_flag'] = 1 if imgrank==1 else 0
    o['attribution'] = "Data provided about the Lens Media Lab Collection are public domain. Rights restrictions may apply to collection objects or images of those objects."
    o['license'] = "https://creativecommons.org/licenses/by/2.0/"

    return o

In [39]:
smp['ImageAssets'] = [[get_image_object(os.path.basename(i)) for i in item] for item in smp.ImagePaths]

In [43]:
del smp['ImagePaths']

In [44]:
smp = smp.reset_index(drop=True)

In [53]:
del smp['Year']

In [54]:
d = smp.to_dict('records')

In [55]:
import json

In [56]:
with open('lml_100sample.jsonl', 'w') as file:
    for item in d:
        json.dump(item, file)
        file.write('\n')

In [57]:
d

[{'ObjectNumber': '104',
  'Manufacturer': 'Kodak',
  'Brand': 'Velox',
  'CreditLine': 'Lens Media Lab Collection',
  'Classification': 'photographic paper package',
  'Department': 'Lens Media Lab',
  'Institution': 'Institute for the Preservation of Cultural Heritage',
  'Date': '1948',
  'ObjectLabel': 'Kodak Velox 1948',
  'ImageAssets': [{'id': '104v',
    'image_filename': '104v.jpg',
    'image_label': 'verso',
    'rank': '2',
    'object_thumbnail_flag': 0,
    'attribution': 'Data provided about the Lens Media Lab Collection are public domain. Rights restrictions may apply to collection objects or images of those objects.',
    'license': 'https://creativecommons.org/licenses/by/2.0/'},
   {'id': '104',
    'image_filename': '104.jpg',
    'image_label': 'recto',
    'rank': '1',
    'object_thumbnail_flag': 0,
    'attribution': 'Data provided about the Lens Media Lab Collection are public domain. Rights restrictions may apply to collection objects or images of those object